#### Reads

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from modules.data_preprocessor import DataPreprocessor
from modules.utils.forecasting import StatisticalForecaster, GlobalForecaster
import lightgbm as lgb
import pandas as pd
from modules.data_preprocessor import DataPreprocessor
from modules.evaluator import ForecastEvaluator
from mlforecast.target_transforms import LocalStandardScaler
from modules.utils.user_configurations import PASSWORDS
from modules.client_config_manager import ConfigManager
from modules.utils.configurations.m5 import DEMO_FORECAST_CONFIGURATIONS, DEMO_MODEL_CONFIGURATIONS, DEMO_BENCHMARK_CONFIGURATIONS, DEMO_EVALUATION_CONFIGURATIONS
from modules.utils.utils import pivoted_df

from modules.forecaster import Forecaster
import xgboost as xgb
# ggplot2 style
plt.style.use('ggplot')
from modules.utils.utils import fill_date_gaps_with_covariates_test_only


from modules.utils.metrics import rmsse, mae, bias

/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/statsforecast/core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Save 
save_path = '/home/filtheo/tetra_product/data/m5_editted/m5_train_df.csv'

df = pd.read_csv(save_path)
df['date'] = pd.to_datetime(df['date'], format='%Y.%m.%d')
df.head()

/tmp/ipykernel_528/3518686884.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(save_path)


,unique_id,date,y,sell_price,event_type_1,event_type_2,snap
0,FOODS_1_001,2014-04-26,2,2.24,NaN,NaN,0.0
1,FOODS_1_002,2014-04-26,0,8.88,NaN,NaN,0.0
2,FOODS_1_003,2014-04-26,0,2.88,NaN,NaN,0.0
3,FOODS_1_004,2014-04-26,16,1.78,NaN,NaN,0.0
4,FOODS_1_005,2014-04-26,0,3.28,NaN,NaN,0.0


In [3]:
# Predict -> assuming hyperparams are tuned
model_config = DEMO_MODEL_CONFIGURATIONS
forecast_config = DEMO_FORECAST_CONFIGURATIONS
benchmark_config = DEMO_BENCHMARK_CONFIGURATIONS
#wednesdays_off_path  =benchmark_config["wednesdays_off_path"]
wednesdays_off_path = "/home/filtheo/tetra_product/modules/utils/configurations/extra_days.csv"

In [4]:
preprocessor = DataPreprocessor(df)
full_data = preprocessor.preprocess(remove_weekends=forecast_config["remove_weekends"],
                                        item_time_covars=forecast_config["item_time_covars"],
                                        filter_dates=True,
                                        )

In [5]:
# load stat preds
path = "/home/filtheo/TETRA_AI_SMP/TETRA/02_Product/second_run/results_folder/m5_stats_results.csv"
stat_preds = pd.read_csv(path)
stat_preds.head()


,unique_id,date,cutoff,True,Model,y,cv,fh
0,FOODS_1_001,2016-02-18,2016-02-17,0.0,Naive,3.0,1,1
1,FOODS_1_001,2016-02-19,2016-02-17,0.0,Naive,3.0,1,2
2,FOODS_1_001,2016-02-20,2016-02-17,0.0,Naive,3.0,1,3
3,FOODS_1_001,2016-02-21,2016-02-17,1.0,Naive,3.0,1,4
4,FOODS_1_001,2016-02-22,2016-02-17,0.0,Naive,3.0,1,5


In [6]:
metrics = [rmsse, mae, bias]

eval = ForecastEvaluator(freq="D", original_df=full_data)
eval.fit(complete_evaluation_df = stat_preds, evaluation_cv=True)
eval_df = eval.predict(metrics=metrics)
eval_df.groupby('Model').mean().drop(columns = ['cv'])

/tmp/ipykernel_528/2393263696.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  eval_df.groupby('Model').mean().drop(columns = ['cv'])


,rmsse,mae,bias
Model,,,
AutoETS,0.724727,1.241240,0.064836
AutoTheta,0.726305,1.241416,0.086842
CrostonOptimized,0.755651,1.344075,0.106320
Naive,0.920978,1.584720,0.109034
SeasWA,0.771355,1.300303,-0.125756
SeasonalNaive,0.988476,1.532894,0.122045
Stats_Ensemble_Mean,0.735069,1.258022,0.075097
Stats_Ensemble_Median,0.718086,1.225995,0.028889
WindowAverage,0.743412,1.319700,0.162362


#### ML

In [7]:
# Define the configurations
ml_config = dict()

ml_config["lags"] = 7 # one week
ml_config["total_lags"] = list(range(1, ml_config["lags"] + 1))

ml_config['loss'] = 'mae'
ml_config['rs'] = 42
ml_config['future_covariates'] = forecast_config["item_time_covars"]
ml_config["total_covariates"] = forecast_config["item_time_covars"]
ml_config['univariate_columns'] = ['unique_id', 'date', 'y']


In [ ]:
# define the model
forecasting_model = lgb.LGBMRegressor(
                objective=ml_config['loss'],
                random_state=ml_config["rs"],
                verbose=-1,
                )
models = {'lgbm': forecasting_model}


In [25]:
# vanilla model
gb = GlobalForecaster(
            models=models,
        lags=list(range(1, ml_config["lags"] + 1)),
        freq=forecast_config["freq"],
        seasonal_length=None,
        )

# no future covariates
gb.fit(full_data[ml_config['univariate_columns']], static_features=None)
gb_preds = gb.predict(h = 7, cv = 5, holdout = True)

eval = ForecastEvaluator(freq="D", original_df=full_data)
eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
eval_df = eval.predict(metrics=metrics)
eval_df.groupby('Model').mean().drop(columns = ['cv'])

/tmp/ipykernel_2772/2803993077.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  eval_df.groupby('Model').mean().drop(columns = ['cv'])


,rmsse,mae,bias
Model,,,
lgbm,0.68565,1.108969,-0.415434


In [34]:
# with covarites
gb = GlobalForecaster(
            models=models,
        lags=list(range(1, ml_config["lags"] + 1)),
        freq=forecast_config["freq"],
        seasonal_length=None,
        )

# no future covariates
gb.fit(full_data[ml_config['univariate_columns'] + ml_config['future_covariates']], static_features=[])
gb_preds = gb.predict(h = 7, cv = 5, holdout = True, future_covars=forecast_config["item_time_covars"])

eval = ForecastEvaluator(freq="D", original_df=full_data)
eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
eval_df = eval.predict(metrics=metrics)
eval_df.groupby('Model').mean().drop(columns = ['cv'])

/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarn

,rmsse,mae,bias
Model,,,
lgbm,0.686745,1.110606,-0.460766


In [35]:
# with scaling
gb = GlobalForecaster(
            models=models,
        lags=list(range(1, ml_config["lags"] + 1)),
        freq=forecast_config["freq"],
        transformations = [LocalStandardScaler()],
        seasonal_length=None,
        )
gb.fit(full_data[ml_config['univariate_columns']], static_features=None)
gb_preds = gb.predict(h = 7, cv = 5, holdout = True)

eval = ForecastEvaluator(freq="D", original_df=full_data)
eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
eval_df = eval.predict(metrics=metrics)
eval_df.groupby('Model').mean().drop(columns = ['cv'])

/tmp/ipykernel_2772/1633527626.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  eval_df.groupby('Model').mean().drop(columns = ['cv'])


,rmsse,mae,bias
Model,,,
lgbm,0.683613,1.098005,-0.481734


In [37]:
#with future covariates and scaling
gb.fit(full_data[ml_config['univariate_columns'] + ml_config['future_covariates']], static_features=[])
gb_preds = gb.predict(h = 7, cv = 5, holdout = True, future_covars=forecast_config["item_time_covars"])

eval = ForecastEvaluator(freq="D", original_df=full_data)
eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
eval_df = eval.predict(metrics=metrics)
eval_df.groupby('Model').mean().drop(columns = ['cv'])

/tmp/ipykernel_2772/636197003.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  eval_df.groupby('Model').mean().drop(columns = ['cv'])


,rmsse,mae,bias
Model,,,
lgbm,0.684371,1.098917,-0.487807


In [ ]:
# pick the best loss with future covariates
losses = ['mse', 'fair', 'mape', 'mae', 'tweedie']
loss_dict = {}
for loss in losses:
    ml_config['loss'] = loss
    forecasting_model = lgb.LGBMRegressor(
                objective=ml_config['loss'],
                random_state=ml_config["rs"],
                verbose=-1,
                )
    models = {'lgbm': forecasting_model}
    gb = GlobalForecaster(
                models=models,
            lags=list(range(1, ml_config["lags"] + 1)),
            freq=forecast_config["freq"],
            seasonal_length=None,
            )

    gb.fit(full_data[ml_config['univariate_columns'] + ml_config['future_covariates']], static_features=[])
    gb_preds = gb.predict(h = 7, cv = 5, holdout = True, future_covars=forecast_config["item_time_covars"])

    eval = ForecastEvaluator(freq="D", original_df=full_data)
    eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
    eval_df = eval.predict(metrics=metrics)
    loss_dict[loss] = eval_df.groupby('Model').mean().drop(columns = ['cv'])

In [40]:
loss_dict

{'mse':           rmsse       mae      bias
 Model                              
 lgbm   0.823631  1.287533  0.180926,
 'fair':           rmsse        mae      bias
 Model                               
 lgbm   3.049173  35.719457  6.135551,
 'mape':           rmsse       mae      bias
 Model                              
 lgbm   0.802947  1.622018 -1.487264,
 'mae':           rmsse       mae      bias
 Model                              
 lgbm   0.686745  1.110606 -0.460766,
 'tweedie':          rmsse       mae      bias
 Model                             
 lgbm   0.79322  1.278537  0.156893}

In [42]:
# pick the best loss without future covariates and scaling
losses = ['mse', 'fair', 'mape', 'mae']
loss_dict = {}
for loss in losses:
    ml_config['loss'] = loss
    forecasting_model = lgb.LGBMRegressor(
                objective=ml_config['loss'],
                random_state=ml_config["rs"],
                verbose=-1,
                )
    models = {'lgbm': forecasting_model}
    gb = GlobalForecaster(
                models=models,
            lags=list(range(1, ml_config["lags"] + 1)),
            freq=forecast_config["freq"],
            seasonal_length=None,
            transformations = [LocalStandardScaler()],
            )

    gb.fit(full_data[ml_config['univariate_columns']], static_features=None)
    gb_preds = gb.predict(h = 7, cv = 5, holdout = True)

    eval = ForecastEvaluator(freq="D", original_df=full_data)
    eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
    eval_df = eval.predict(metrics=metrics)
    loss_dict[loss] = eval_df.groupby('Model').mean().drop(columns = ['cv'])

/tmp/ipykernel_2772/3067316082.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  loss_dict[loss] = eval_df.groupby('Model').mean().drop(columns = ['cv'])
/tmp/ipykernel_2772/3067316082.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  loss_dict[loss] = eval_df.groupby('Model').mean().drop(columns = ['cv'])
/tmp/ipykernel_2772/3067316082.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  loss_dict[loss] = eval_

In [43]:
loss_dict

{'mse':           rmsse       mae     bias
 Model                             
 lgbm   0.691741  1.204776  0.08642,
 'fair':           rmsse       mae      bias
 Model                              
 lgbm   0.664996  1.125609 -0.164781,
 'mape':           rmsse       mae     bias
 Model                             
 lgbm   0.696852  1.108752 -0.63615,
 'mae':           rmsse       mae      bias
 Model                              
 lgbm   0.683613  1.098005 -0.481734}

In [ ]:
# we have two options.
# either fair with no future covariates and scaling
# or mae with future covariates and scaling
# conflict on two losess!
# MAE underestimates and MSE overestimates

In [45]:
# seasonal features
import itertools
seasonal_features = ['dayofweek', 'month', 'day',]
# All possible combinations 
combinations = []
for r in range(1, len(seasonal_features) + 1):
    combinations.extend(itertools.combinations(seasonal_features, r))
# add empty list
combinations.append(())

In [46]:
seasonal_dict = {}
for comb in combinations:
    # first mae and future covariates
    forecasting_model = lgb.LGBMRegressor(
                objective='mae',
                random_state=ml_config["rs"],
                verbose=-1,
            )
    models = {'lgbm': forecasting_model}
    gb = GlobalForecaster(
                models=models,
            lags=list(range(1, ml_config["lags"] + 1)),
            freq=forecast_config["freq"],
            seasonal_length=None,
            date_features =comb,
            )
    gb.fit(full_data[ml_config['univariate_columns'] + ml_config['future_covariates']], static_features=[])
    gb_preds = gb.predict(h = 7, cv = 5, holdout = True, future_covars=forecast_config["item_time_covars"])
    eval = ForecastEvaluator(freq="D", original_df=full_data)
    eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
    eval_df = eval.predict(metrics=metrics)
    seasonal_dict[comb] = eval_df.groupby('Model').mean().drop(columns = ['cv'])

/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarning: Found null values in event_type_1, event_type_2.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/home/filtheo/Cloud-for-AI-VIVES/.venv2/lib/python3.8/site-packages/mlforecast/core.py:592: UserWarn

In [47]:
seasonal_dict

{('dayofweek',):           rmsse       mae      bias
 Model                              
 lgbm   0.680251  1.104436 -0.415246,
 ('month',):           rmsse      mae      bias
 Model                             
 lgbm   0.687305  1.10942 -0.469009,
 ('day',):           rmsse       mae      bias
 Model                              
 lgbm   0.686626  1.109329 -0.462726,
 ('dayofweek',
  'month'):           rmsse       mae      bias
 Model                              
 lgbm   0.680671  1.106181 -0.425797,
 ('dayofweek',
  'day'):          rmsse      mae      bias
 Model                            
 lgbm   0.68066  1.10101 -0.430481,
 ('month',
  'day'):           rmsse       mae      bias
 Model                              
 lgbm   0.687226  1.109363 -0.470734,
 ('dayofweek',
  'month',
  'day'):           rmsse       mae      bias
 Model                              
 lgbm   0.679788  1.104253 -0.434504,
 ():           rmsse       mae      bias
 Model                              
 lgb

In [48]:
seasonal_dict = {}
for comb in combinations:
    # now fair and scaling
    forecasting_model = lgb.LGBMRegressor(
                objective='fair',
                random_state=ml_config["rs"],
                verbose=-1,
            )
    models = {'lgbm': forecasting_model}
    gb = GlobalForecaster(
                models=models,
            lags=list(range(1, ml_config["lags"] + 1)),
            freq=forecast_config["freq"],
            seasonal_length=None,
            date_features =comb,
            transformations = [LocalStandardScaler()],
            )
    gb.fit(full_data[ml_config['univariate_columns']], static_features=None)
    gb_preds = gb.predict(h = 7, cv = 5, holdout = True)
    eval = ForecastEvaluator(freq="D", original_df=full_data)
    eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
    eval_df = eval.predict(metrics=metrics)
    seasonal_dict[comb] = eval_df.groupby('Model').mean().drop(columns = ['cv'])

/tmp/ipykernel_2772/1197445107.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_dict[comb] = eval_df.groupby('Model').mean().drop(columns = ['cv'])
/tmp/ipykernel_2772/1197445107.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_dict[comb] = eval_df.groupby('Model').mean().drop(columns = ['cv'])
/tmp/ipykernel_2772/1197445107.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_dict[c

In [49]:
seasonal_dict

{('dayofweek',):          rmsse       mae      bias
 Model                             
 lgbm   0.66153  1.109395 -0.165917,
 ('month',):           rmsse       mae      bias
 Model                              
 lgbm   0.665913  1.131255 -0.129977,
 ('day',):           rmsse       mae      bias
 Model                              
 lgbm   0.665219  1.113284 -0.285052,
 ('dayofweek',
  'month'):          rmsse       mae      bias
 Model                             
 lgbm   0.66238  1.114027 -0.128531,
 ('dayofweek',
  'day'):           rmsse      mae      bias
 Model                             
 lgbm   0.662423  1.09852 -0.297608,
 ('month',
  'day'):           rmsse       mae      bias
 Model                              
 lgbm   0.665614  1.116467 -0.267883,
 ('dayofweek',
  'month',
  'day'):           rmsse       mae      bias
 Model                              
 lgbm   0.662137  1.101654 -0.251141,
 ():           rmsse       mae      bias
 Model                              
 lgb

In [ ]:
# Its kind of close
# For simplicity I will go with fair and scaling
# We extra tuning we could make the selection
# The optimal solution is to optimize both and get their ensemble!

In [51]:
forecasting_model = lgb.LGBMRegressor(
                objective='fair',
                random_state=ml_config["rs"],
                verbose=-1,
            )
models = {'lgbm': forecasting_model}
date_features = ['dayofweek', 'month', 'day']

In [52]:
# Lags
lag_range = np.arange(7, 28) # one to four weeks
lags_dict = dict()
for lag in lag_range:
    ml_config["lags"] = lag
    ml_config["total_lags"] = list(range(1, lag + 1))
    gb = GlobalForecaster(
            models=models,
            lags=ml_config["total_lags"],
            freq=forecast_config["freq"],
            seasonal_length=None,
            date_features = date_features,
            transformations = [LocalStandardScaler()],
            )
    gb.fit(full_data[ml_config['univariate_columns']], static_features=None)
    gb_preds = gb.predict(h = 7, cv = 5, holdout = True)
    eval = ForecastEvaluator(freq="D", original_df=full_data)
    eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
    eval_df = eval.predict(metrics=metrics)
    lags_dict[lag] = eval_df.groupby('Model').mean().drop(columns = ['cv'])

/tmp/ipykernel_2772/3825658577.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  lags_dict[lag] = eval_df.groupby('Model').mean().drop(columns = ['cv'])
/tmp/ipykernel_2772/3825658577.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  lags_dict[lag] = eval_df.groupby('Model').mean().drop(columns = ['cv'])
/tmp/ipykernel_2772/3825658577.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  lags_dict[lag] = eval_df.

In [54]:
# Sort dictionary by rmsse values
sorted_lags = dict(sorted(lags_dict.items(), key=lambda x: x[1].loc['lgbm', 'rmsse']))
sorted_lags

{21:           rmsse       mae      bias
 Model                              
 lgbm   0.651302  1.089545 -0.210627,
 23:           rmsse       mae      bias
 Model                              
 lgbm   0.651698  1.091003 -0.208318,
 25:           rmsse       mae     bias
 Model                             
 lgbm   0.651734  1.089901 -0.21206,
 22:          rmsse     mae     bias
 Model                          
 lgbm   0.65182  1.0904 -0.20891,
 24:           rmsse       mae     bias
 Model                             
 lgbm   0.651861  1.090274 -0.21183,
 26:           rmsse       mae      bias
 Model                              
 lgbm   0.651883  1.089973 -0.210431,
 27:           rmsse       mae      bias
 Model                              
 lgbm   0.651929  1.088824 -0.214883,
 19:           rmsse       mae      bias
 Model                              
 lgbm   0.652017  1.091303 -0.210537,
 20:           rmsse       mae      bias
 Model                              
 lgbm   0.65

In [8]:
# Final model
forecasting_model = lgb.LGBMRegressor(
                objective='fair',
                random_state=ml_config["rs"],
                verbose=-1,
            )
models = {'lgbm': forecasting_model}
date_features = ['dayofweek', 'month', 'day']
lags = 21
gb = GlobalForecaster(
            models=models,
        lags=list(range(1, lags + 1)),
        freq=forecast_config["freq"],
        seasonal_length=None,
        date_features = date_features,
        transformations = [LocalStandardScaler()],
        )

gb.fit(full_data[ml_config['univariate_columns']], static_features=None)
gb_preds = gb.predict(h = 7, cv = 5, holdout = True)
eval = ForecastEvaluator(freq="D", original_df=full_data)
eval.fit(complete_evaluation_df = gb_preds, evaluation_cv=True)
eval_df = eval.predict(metrics=metrics)


#### Ensembling

In [10]:
import itertools


In [28]:
#statts_preds_copy = stat_preds.copy()# all combinations of the unique models on Model column
models = stat_preds['Model'].unique()

# for simplicity I am only taking three models
models = ['AutoETS', 'SeasonalNaive', 'SeasWA']
# Generate all possible combinations of models
all_combinations = [
    list(combo) for r in range(1, len(models) + 1)
    for combo in itertools.combinations(models, r)
]

In [29]:
for comb in all_combinations:
    statts_preds_copy = stat_preds[stat_preds['Model'].isin(comb)]

    preds = pd.concat([gb_preds, statts_preds_copy])
    preds['date'] = pd.to_datetime(preds['date'])
    preds['cutoff'] = pd.to_datetime(preds['cutoff'])
    cols = ['unique_id', 'date', 'cutoff', 'True', 'cv', 'fh']

    ensemble_preds = preds.groupby(cols).median(numeric_only=True).reset_index()
    ensemble_preds['Model'] = 'Ensemble'
    
    eval = ForecastEvaluator(freq="B", original_df=full_data)
    eval.fit(complete_evaluation_df = ensemble_preds, evaluation_cv=True)
    eval_df = eval.predict(metrics=metrics)
    print(comb)
    print(eval_df.groupby('Model').mean(numeric_only=True))

['AutoETS']
             rmsse       mae      bias   cv
Model                                      
Ensemble  0.650777  1.100596 -0.073953  3.0
['SeasonalNaive']
            rmsse       mae      bias   cv
Model                                     
Ensemble  0.72529  1.179037 -0.047637  3.0
['SeasWA']
             rmsse       mae      bias   cv
Model                                      
Ensemble  0.667847  1.120124 -0.161622  3.0
['AutoETS', 'SeasonalNaive']
             rmsse       mae      bias   cv
Model                                      
Ensemble  0.656972  1.094885 -0.092323  3.0
['AutoETS', 'SeasWA']
             rmsse       mae      bias   cv
Model                                      
Ensemble  0.657507  1.108339 -0.107629  3.0
['SeasonalNaive', 'SeasWA']
             rmsse       mae     bias   cv
Model                                     
Ensemble  0.669976  1.100753 -0.18418  3.0
['AutoETS', 'SeasonalNaive', 'SeasWA']
             rmsse       mae      bias   cv
Model      

In [ ]:
# Ensembling with ETS is the go to!
# however ETS is pretty slow to get forecasts.
# I will skip for now!
# remember, there are always tradeoffs!

# We update the final model on the configurations
